<a href="https://colab.research.google.com/github/PeterRoumeliotis/AIFireSmokeDetectionResearchProject/blob/main/AIFireAndSmokeDetectionResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import shutil
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Dense, Dropout
import cv2
import numpy as np
import time
import kagglehub
from google.colab import files

In [5]:
# Downloading latest version of the dataset I am using
path = kagglehub.dataset_download("phylake1337/fire-dataset")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fire-dataset


In [6]:
random.seed(42)

# Getting the source paths
source_dir = '/kaggle/input/fire-dataset/fire_dataset'
source_fire_dir = os.path.join(source_dir, 'fire_images')
source_non_fire_dir = os.path.join(source_dir, 'non_fire_images')

# Creating training and validation paths
dest_dir = 'data'
train_fire_dir = os.path.join(dest_dir, 'train', 'fire')
val_fire_dir = os.path.join(dest_dir, 'validation', 'fire')
train_non_fire_dir = os.path.join(dest_dir, 'train', 'non_fire')
val_non_fire_dir = os.path.join(dest_dir, 'validation', 'non_fire')

# Making sure the directories exist and if they dont, making them
os.makedirs(train_fire_dir, exist_ok=True)
os.makedirs(val_fire_dir, exist_ok=True)
os.makedirs(train_non_fire_dir, exist_ok=True)
os.makedirs(val_non_fire_dir, exist_ok=True)

# 80% training 20% validation
split_ratio = 0.8

# Splitting the files into each folder
def split_data(source_folder, train_folder, val_folder, split_ratio=0.8):

    # List all items in source folder and keep only files
    file_list = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    # Randomizes it
    random.shuffle(file_list)

    # Figures out at what index to split the files
    split_point = int(len(file_list) * split_ratio)
    train_files = file_list[:split_point]   # Training
    val_files = file_list[split_point:]   # Validation

    # Copying files into training
    for file_name in train_files:
        src = os.path.join(source_folder, file_name)
        dst = os.path.join(train_folder, file_name)
        shutil.copy(src, dst)

    # Copying files into validation
    for file_name in val_files:
        src = os.path.join(source_folder, file_name)
        dst = os.path.join(val_folder, file_name)
        shutil.copy(src, dst)

split_data(source_fire_dir, train_fire_dir, val_fire_dir, split_ratio)
split_data(source_non_fire_dir, train_non_fire_dir, val_non_fire_dir, split_ratio)

print("Data is split")

Data is split


In [7]:
img_width, img_height = 150, 150

batch_size = 32
epochs = 20

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

# Augmenting training data
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Rescaling validation data
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Generate validation batches
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 799 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


# My Model

In [8]:
from tensorflow.keras.optimizers import Adam
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 148, 148, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 72, 72, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 34, 34, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,785 (432.75 KB)

 Trainable params: 110,337 (431.00 KB)

 Non-trainable params: 448 (1.75 KB)

In [9]:
history = model.fit(train_generator, steps_per_epoch=train_generator.samples, epochs=epochs, validation_data=validation_generator, validation_steps=validation_generator.samples // batch_size)

model_save_path = "fire_detection_cnn.keras"
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 25/799 ━━━━━━━━━━━━━━━━━━━━ 9:36 745ms/step - accuracy: 0.8719 - loss: 0.2906

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


799/799 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - accuracy: 0.9014 - loss: 0.2319 - val_accuracy: 0.7656 - val_loss: 0.5687
Epoch 2/20
799/799 ━━━━━━━━━━━━━━━━━━━━ 31s 23ms/step - accuracy: 0.9358 - loss: 0.1666 - val_accuracy: 0.7552 - val_loss: 0.5132
Epoch 3/20
799/799 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.9450 - loss: 0.1414 - val_accuracy: 0.7448 - val_loss: 0.4877
Epoch 4/20
799/799 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.9412 - loss: 0.1392 - val_accuracy: 0.7552 - val_loss: 0.4277
Epoch 5/20
799/799 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9257 - loss: 0.1775 - val_accuracy: 0.7552 - val_loss: 0.4070
Epoch 6/20
799/799 ━━━━━━━━━━━━━━━━━━━━ 19s 23ms/step - accuracy: 0.9378 - loss: 0.1653 - val_accuracy: 0.7604 - val_loss: 0.4021
Epoch 7/20
799/799 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.9516 - loss: 0.1165 - val_accuracy: 0.7500 - val_loss: 0.5408
Epoch 8/20
799/799 ━━━━━━━━━━━━━━━━━━━━ 22s 26ms/step - accuracy: 0.9423 - loss: 0.1425 - val_accurac

In [10]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.5/983.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 836.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.6 MB/s eta 0:00:00


In [16]:
import os

# https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset/
# https://www.digitalocean.com/community/tutorials/train-yolov5-custom-data
# Sources I used to help me format my dataset for yolo

for split in ["train","validation"]:
    for cls_idx, cls in enumerate(["non_fire","fire"]):
        folder = f"data/{split}/{cls}"
        for img in os.listdir(folder):
            # Skips non images
            if not img.lower().endswith((".jpg",".png")): continue
            img_path   = os.path.join(folder, img)
            label_path = os.path.splitext(img_path)[0] + ".txt"

            if cls == "fire":
                with open(label_path, "w") as f:
                    # Formats data for yolo
                    f.write(f"{1} 0.5 0.5 1.0 1.0\n")
            else:
                open(label_path, "w").close()


In [17]:
# Making YAML for yolo

# https://docs.ultralytics.com/
# https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/

# Used these sources for help

%%bash
cat <<EOF > data/fire_data.yaml
train: /content/data/train
val:   /content/data/validation

names:
  0: non_fire
  1: fire
EOF

echo "Created YAML:"
cat data/fire_data.yaml


Created YAML:
train: /content/data/train
val:   /content/data/validation

names:
  0: non_fire
  1: fire


In [13]:
from ultralytics import YOLO

yolo = YOLO('yolov8n.pt')

yolo.train(
    data='data/fire_data.yaml',
    epochs=20,
    imgsz=150,
    batch=32,
    project='yolo-fire',
    name='exp'
)

yolo_model = YOLO('yolo-fire/exp/weights/best.pt')


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 107MB/s]


Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data/fire_data.yaml, epochs=20, time=None, patience=100, batch=32, imgsz=150, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolo-fire, name=exp, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None

100%|██████████| 755k/755k [00:00<00:00, 18.6MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      20/20      0.68G     0.0932     0.1746     0.8476         27        160: 100%|██████████| 25/25 [00:09<00:00,  2.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.89it/s]

                   all        200        151      0.979      0.993      0.994      0.994



20 epochs completed in 0.083 hours.
Optimizer stripped from yolo-fire/exp/weights/last.pt, 6.2MB
Optimizer stripped from yolo-fire/exp/weights/best.pt, 6.2MB

Validating yolo-fire/exp/weights/best.pt...
Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.84it/s]


                   all        200        151      0.993      0.986      0.995      0.995
                  fire        151        151      0.993      0.986      0.995      0.995
Speed: 0.0ms preprocess, 0.5ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to yolo-fire/exp


In [18]:
uploaded = files.upload()
video_path = next(iter(uploaded.keys()))
print("Video file:", video_path)

# https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html
# https://www.geeksforgeeks.org/time-perf_counter-function-in-python/
# https://docs.ultralytics.com/usage/python/
# Used these sources to help me learn this

# Loading my model
cnn_model = tf.keras.models.load_model('fire_detection_cnn.keras')

# OpenCV opening my video of a fire starting
cap = cv2.VideoCapture(video_path)

# Frames per second (if not I set it to 30)
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

# Setting up arrays
cnn_times, yolo_times = [], []
cnn_detections, yolo_detections = [], []

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1
    timestamp = frame_idx / fps

    # CNN
    t0 = time.perf_counter()
    img = cv2.resize(frame, (150,150)) / 255.0
    p = cnn_model.predict(np.expand_dims(img,0), verbose=0)[0][0]
    t1 = time.perf_counter()
    cnn_times.append(t1 - t0)
    if p > 0.5 and not cnn_detections:
        cnn_detections.append(timestamp)

    # YOLOv8
    t2 = time.perf_counter()
    results = yolo_model(frame, verbose=False)
    t3 = time.perf_counter()
    yolo_times.append(t3 - t2)
    for r in results:
        if r.boxes.cls.cpu().numpy().tolist().count(1) > 0 and not yolo_detections:
            yolo_detections.append(timestamp)
            break

cap.release()

import numpy as _np
print(f"CNN first detection at {cnn_detections or ['never']} seconds")
print(f"YOLO first detection at {yolo_detections or ['never']} seconds")
print(f"Avg CNN frame time: {_np.mean(cnn_times):.4f}s")
print(f"Avg YOLO frame time: {_np.mean(yolo_times):.4f}s")


Saving videoplayback.mp4 to videoplayback (1).mp4
Video file: videoplayback (1).mp4
CNN first detection at [0.03336666666666667] seconds
YOLO first detection at [0.03336666666666667] seconds
Avg CNN frame time: 0.0665s
Avg YOLO frame time: 0.0102s
